In [ ]:
! pip install --quiet environs cyksuid toolz psycopg2-binary typing_json backoff xxhash pyyaml geopandas dtreeviz facebook_business

In [1]:
%load_ext autoreload
%autoreload 2

In [12]:
from environs import Env
env = Env()
env.read_env('.env-upswell-vlab', override=True)

In [ ]:
from adopt.malaria import get_conf, get_df, lookalike, get_ad_token
from adopt.marketing import Marketing
from adopt.facebook.state import CampaignState

cnf = get_conf(env)
token = get_ad_token(cnf["survey_user"], cnf["chatbase"])

state = CampaignState(env, token)
m = Marketing(env, state)

# TODO: make new strata format conf parse
# TODO: make get_df work with new strata format
# df = get_df(cnf)

In [ ]:
from adopt.marketing import load_strata_conf


load_strata_conf('config/strata-new.json')

In [24]:
from adopt.malaria import load_typed_json
from adopt.marketing import Audience
from typing import List

load_typed_json("config/audiences.json", List[Audience])

[Audience(name='kutcha', type='lookalike', shortcodes=[], target_questions=[], lookalike_spec=LookalikeSpec(country='IN', ratio=0.1, starting_ratio=0.0)),
 Audience(name='respondents', type='retargeting', shortcodes=[], target_questions=[], lookalike_spec=None)]

In [ ]:
state.custom_audiences

In [ ]:
sorted(state.custom_audiences, key=lambda x: x["time_created"])

In [12]:
len({ca["name"] for ca in state.custom_audiences})

23

In [ ]:
from adopt.malaria import create_lookalike

create_lookalike(cnf, m)

In [5]:
import logging

logging.getLogger().setLevel(logging.ERROR)

In [6]:
from adopt.malaria import update_respondent_audience
update_respondent_audience(cnf, df, m)

In [7]:
import geopandas as gpd
from adopt.malaria import load_cities

def get_dist_info(path):
    geod = gpd.read_file(path)
    district_info = geod.groupby('disthash').apply(lambda df: df.iloc[0][['disthash', 'tot_p']]).reset_index(drop=True)
    return district_info

dist_info = get_dist_info('outs/cluster.shp')
cities = load_cities('output/cities-22.csv')
cities = cities.merge(dist_info, how='left', on='disthash')

districts = cities.groupby('distname') \
                  .head(1) \
                  .reset_index(drop=True) \
                  .drop(columns=['rad', 'lng', 'lat', 'distcode', 'id'])

In [8]:
from adopt.responses import get_forms
from datetime import datetime

survey_user = '10383123-9fb1-429b-8312-447c1b54b17a'
shortcodes = [ 'follow1eng', 'follow1hin']
# shortcodes = [ 'baselineeng', 'baselinehin', 'baselinehinexc']

timestamp = datetime(2020, 9, 1)

forms = list(get_forms(survey_user, shortcodes, timestamp, cnf['chatbase']))

In [9]:
questions =[('malaria_now', '58583e72-7036-4c74-9ebd-179598cba59f'),
             ('net_lastnight', '1f812d37-a7a5-400d-8a31-145fbb49f577'),]

In [15]:
questions = [
    ('dist_medical', 'd7573919-8a7e-457f-9a1d-1f8c389127a7'),
    ('education', 'e40fa1c6-13a1-4a02-91cd-0eaade11864d'),
    ('owns_net', 'bd4802c6-7bdb-40f0-aac1-18cc6df7da6e'),
    ('malaria_incidence', 'f6e69027-97cc-494e-8d52-318b75047e23'),
    ('malaria_now', 'dad23031-8468-4900-89cc-d01841d8b660'),
    ('net_lastnight', 'b5b1ff58-c8df-4890-9b1c-0cd40ce6edc0'),
    ('home', '4d0ae478-3893-4b46-ab39-d6848c69245d'), 
    ('occupation', '4fc929c7-132d-49b1-a164-515e5cc9064f')]

In [10]:
from adopt.responses import get_response_df
from datetime import datetime, timezone
from adopt.clustering import only_target_users
import pandas as pd
from adopt.clustering import add_res_cols
from adopt.forms import response_translator
from adopt.malaria import shape_df
import re


def filter_time(df, min_date, max_date):
    min_date = datetime(*min_date, tzinfo=timezone.utc)
    max_date = datetime(*max_date, tzinfo=timezone.utc)

    users = df[(df.timestamp > min_date) & (df.timestamp < max_date)].userid.unique()
    return df[df.userid.isin(users)]

def ref_translation(eng_form, other_form):
    vals = [(a['ref'], b['ref']) for a, b 
            in zip(eng_form['fields'], other_form['fields'])]
    lookup = dict(vals)
    rev_lookup = {v:k for k,v in lookup.items()}
    return lookup, rev_lookup


def get_filtered_responses(survey_user, eng_form, other_form, shortcodes, questions, db_cnf):
    ref_lookup, rev_ref_lookup = ref_translation(eng_form, other_form)
    refs = [ref_lookup[r] for _, r in questions]
    responses = get_response_df(survey_user, shortcodes, refs, db_cnf)

    _, last_ref = questions[-1]
    
    answered = responses[responses.question_ref == last_ref].userid.unique()    

    return responses[responses.userid.isin(answered)].reset_index(drop=True)

def malaria_prob(groupby, key='malaria'):
    d = groupby[key].value_counts(normalize=True)
    d.name = 'count'
    d = d.reset_index(level=-1)
    d = d[d[key] == True].drop(columns=key)
    return d


def stats(df):
    targets = df.kutcha.sum()
    tot = df.shape[0]
    non_target = tot - targets
    return pd.Series([targets/tot, targets, non_target, tot], index=['perc', 'target', 'non_target', 'tot'])


def col_translators(a, b, questions):
    lookup, _ = ref_translation(a, b)

    refs = [ref for _, ref in questions]
    ts = { qt['ref']: response_translator(q, qt)
           for qt, q in zip(a['fields'], b['fields'])
           if qt['ref'] in refs}
    
    return [(n, lookup[ref], ts[ref]) for n, ref in questions]


In [16]:
forms[1]['field']

[{'id': 'HUwXLOovcxw7',
  'title': 'कोई सही या गलत उत्तर नहीं हैं। हम सिर्फ यह जानना चाहते हैं कि आप क्या सोचते हैं! \nध्यान रखें कि आपके उत्तर गोपनीय रहेंगे।',
  'ref': '0ebdd97f-0529-412a-a164-b1a72f1410a2',
  'properties': {'button_text': 'Continue', 'hide_marks': False},
  'type': 'statement'},
 {'id': 'wFLxq0nx4mIX',
  'title': 'क्या आप पिछली रात मच्छरदानी के अंदर सोए थे?',
  'ref': '1f812d37-a7a5-400d-8a31-145fbb49f577',
  'properties': {'randomize': False,
   'allow_multiple_selection': False,
   'allow_other_choice': False,
   'vertical_alignment': True,
   'choices': [{'id': 'zY5ezN2bPleB',
     'ref': '72c28aec-2452-44e6-b7fe-a663157a5c0f',
     'label': 'हाँ'},
    {'id': '6neTf6ghd1x1',
     'ref': 'c761a82d-0b8c-4fe0-857e-a500dd657bf7',
     'label': 'नहीं'}]},
  'validations': {'required': False},
  'type': 'multiple_choice'},
 {'id': 'M1o8IAD2mL0Y',
  'title': 'आपके परिवार में जितने लोग रहते हैं (आप सहित), उसमें से कितने सदस्य कल रात मच्छरदानी के नीचे सोए थे? [नंबर डालें

In [11]:
filtered_res = get_filtered_responses(survey_user, forms[0], forms[1], ['follow1hin'], questions, cnf['chatbase'])
# filtered_res = get_filtered_responses(survey_user, forms[0], forms[1], ['baselinehin', 'baselinehinexc'], questions, cnf['chatbase'])

col_names = col_translators(forms[0], forms[1], questions)
rr = add_res_cols(col_names, shape_df(filtered_res)) \
    .reset_index(drop=True) \
    .dropna(subset=[r for r, _ in questions if not re.match('if_', r)])

AttributeError: 'NoneType' object has no attribute 'question_ref'

In [ ]:
# choices: 
#   ordered: true
#   alphabetize: true
#   booleanize: Yes

In [12]:
dd = rr.merge(districts, left_on='md:clusterid', right_on='disthash')
qrefs = [c for c in dd.columns if len(c) > 35]
dd = dd.drop(columns=qrefs)

dd['under_net'] = dd.net_lastnight == 'Yes'
# dd['malaria'] = dd.malaria_incidence == 'Yes'
dd['malaria_now'] = dd.malaria_now == 'Yes'
# dd['kutcha'] = dd.home == 'Kutcha (made of mud, tin, straw)'
# dd['pucca'] = dd.home == 'Pucca (have cement/brick wall and floor'
# dd['university'] = dd.education == 'University degree or higher'
# dd['unemployed'] = dd.occupation == 'Unemployed'
dd['disthash'] = dd['md:clusterid']

NameError: name 'rr' is not defined

In [13]:
dd.shape

NameError: name 'dd' is not defined

In [19]:
perc = dd.groupby('disthash').apply(stats).reset_index()

In [76]:
# treatment_assignment = pd.read_csv("outs/ma-with-treatment.csv")[['disthash', 'treatment']]
# dd.merge(treatment_assignment).to_csv('outs/individual-with-treatment.csv', index=False)

In [53]:
filter_time(dd, (2020,8,24), (2020,8,25)).shape

(160, 19)

In [54]:
filter_time(dd, (2020,8,25), (2020,8,26)).shape

(204, 19)

In [55]:
filter_time(dd, (2020,8,26), (2020,8,27)).shape

(57, 19)

In [56]:
filter_time(dd, (2020,8,27), (2020,8,28)).shape

(223, 19)

In [57]:
filter_time(dd, (2020,8,28), (2020,8,29)).shape

(1072, 19)

In [58]:
filter_time(dd, (2020,8,29), (2020,8,30)).shape

(383, 19)

In [59]:
filter_time(dd, (2020,8,30), (2020,9,1)).shape

(212, 19)

In [20]:
from adopt.clustering import get_budget_lookup
from adopt.malaria import window, days_left, get_df, get_cluster_from_adset
from adopt.facebook.state import CampaignState, BudgetWindow

# w = BudgetWindow(datetime(2020,8,10), datetime.now())

w = window(96)
state = CampaignState(env, w)

spend = {get_cluster_from_adset(n): i
         for n, i in state.spend.items()}

In [21]:
budget_lookup, prices = get_budget_lookup(df,
                                          stratum,
                                          float('inf'),
                                          10000,
                                          84,
                                          6,
                                          w,
                                          spend, 
                                          return_price=True)

In [22]:
bb = pd.Series(budget_lookup) / 100

In [ ]:
with pd.option_context('display.max_rows', None):
    display(bb.sort_values()[:80])

In [23]:
bb.sort_values()[:80].sum() / 70 * 12

1460.1925714285717

In [24]:
def add_budget(r, lookup, key='budget'):
    r[key] = lookup.get(r['disthash'], None)
    return r

perc = perc.apply(lambda r: add_budget(r, budget_lookup), 1)
perc = perc.apply(lambda r: add_budget(r, prices, 'price'), 1)

In [25]:
mn_lookup = dd.groupby('disthash').apply(lambda df: pd.Series({k: df[k].mean() for k in ['malaria_now', 'malaria']})).reset_index()
perc = perc.merge(mn_lookup)

In [26]:
excluded = cities[cities.creative_group == 'exclusions'].disthash.unique()
perc['exclusion'] = perc.disthash.isin(excluded)

In [27]:
audienced = cities[cities.include_audience == True].disthash.unique()
perc['audienced'] = perc.disthash.isin(audienced)

In [28]:
perc = perc.merge(dd.groupby('disthash').under_net.mean().reset_index(), how='left')

In [29]:
saturated = (only_target_users(df, surveys, 'target_questions')
             .groupby('md:clusterid')
             .userid.count() >= 250) \
             .reset_index() \
             .pipe(lambda df: df[df.userid == True])['md:clusterid']

In [30]:
# cities.loc[cities.disthash == 'c12daf4e', 'creative_group'] = 'exclusions'

In [31]:
final_clusters = pd.read_csv('outs/final-clusters.csv').disthash

In [32]:
perc['final'] = perc.disthash.isin(final_clusters)

In [33]:
with pd.option_context('display.max_rows', None, 'display.expand_frame_repr', False):
    display(perc.sort_values('tot'))

,disthash,perc,target,non_target,tot,budget,price,malaria_now,malaria,exclusion,audienced,under_net,final
96,e1e8a3c2,0.333333,2.0,4.0,6.0,NaN,NaN,0.000000,0.500000,False,False,0.333333,False
74,b52c3191,0.125000,1.0,7.0,8.0,NaN,NaN,0.000000,0.250000,False,False,0.875000,False
69,aa2e27a9,0.375000,3.0,5.0,8.0,NaN,NaN,0.000000,0.250000,False,False,0.625000,False
44,5b86fd59,0.300000,3.0,7.0,10.0,NaN,NaN,0.100000,0.400000,False,False,0.600000,False
19,282c9c61,0.700000,7.0,3.0,10.0,NaN,NaN,0.000000,0.500000,False,False,0.700000,False
67,a5ef19d0,0.500000,5.0,5.0,10.0,NaN,NaN,0.000000,0.200000,False,False,0.900000,False
14,19f0ff06,0.233333,7.0,23.0,30.0,NaN,NaN,0.033333,0.166667,False,False,0.400000,False
0,06622dd4,0.322581,10.0,21.0,31.0,NaN,NaN,0.064516,0.225806,False,False,0.451613,False
7,0cb3e7c3,0.162162,6.0,31.0,37.0,NaN,NaN,0.054054,0.189189,False,False,0.756757,False
70,ae1ad2d3,0.288889,13.0,32.0,45.0,NaN,NaN,0.022222,0.244444,False,False,0.822222,False


In [63]:
treatment_assignment = pd.read_csv("outs/ma-with-treatment.csv")[['disthash', 'treatment']]
# dd.merge(treatment_assignment).to_csv('outs/individual-with-treatment.csv', index=False)

In [70]:
perc = dd.groupby('disthash').apply(lambda df: pd.Series({'tot': df.shape[0]})).reset_index()
treatment_assignment = pd.read_csv("outs/ma-with-treatment.csv")[['disthash', 'treatment']]
perc = perc.merge(treatment_assignment)

In [ ]:
with pd.option_context('display.max_rows', None, 'display.expand_frame_repr', False):
    display(perc.sort_values('tot'))

In [64]:
from adopt.malaria import get_cluster_from_adset

def matching_stats(df):
    vals = {
        'kutchas': df.kutcha.mean(),
        'puccas': df.pucca.mean(),
        'university': df.university.mean(),
        'unemployed': df.unemployed.mean(),
        'malaria': df.malaria.mean(),
        'malaria_now': df.malaria_now.mean(),
        'under_net': df.under_net.mean(),
        'population': df.tot_p.max()
    }

    return pd.Series(vals)

def prep_facebook_data(path, dd, final_clusters):
    capture_count = dd[(dd.timestamp > datetime(2020,7,23,tzinfo=timezone.utc)) & 
                       (dd.timestamp < datetime(2020,7,24,tzinfo=timezone.utc))] \
        .groupby('disthash') \
        .userid.count() \
        .reset_index() \
        .rename(columns={'userid': 'completed_survey'})

    df = pd.read_csv(path)
    df['disthash'] = df['Ad Set Name'].map(get_cluster_from_adset)
    df = df.merge(capture_count, how='left')
    df = df[df.disthash.isin(final_clusters)].reset_index(drop=True)

    spend = 'Amount Spent (INR)'
    df['cost_per_completion'] = df.apply(lambda r: r[spend] / r.completed_survey, 1)
    df = df.rename(columns = {'CTR (All)': 'CTR', 
                              'Cost per Results': 'cost_per_message', 
                              'CPM (Cost per 1,000 Impressions) (INR)': 'CPM'})

    return df[['disthash', 'cost_per_completion', 'cost_per_message', 'CTR', 'CPM']]

In [69]:
dd = dd.rename(columns={'tot_p_x': 'tot_p'})
final_clusters = pd.read_csv('outs/final-clusters.csv').disthash

fb = prep_facebook_data('outs/fb-export-vlab-1.csv', dd, final_clusters)

ma = dd.groupby('disthash').apply(matching_stats).reset_index()
ma = ma[ma.disthash.isin(final_clusters)]
ma.loc[ma.population.isna(), 'population'] = ma.population.quantile(.2) # random data filling :/

ma = ma.merge(fb)

In [70]:
perc['saturated'] = perc.disthash.isin(saturated)

perc_stats = perc[['disthash', 'tot', 'audienced', 'exclusion', 'price', 'saturated']].rename(columns={'tot': 'current_total'})

treatment_assignment = pd.read_csv('outs/ma-with-treatment.csv')[['disthash', 'treatment']]
perc = perc.merge(treatment_assignment, how='left')

ma.merge(treatment_assignment).merge(perc_stats).to_csv('outs/ma-3.csv', index=False)

In [ ]:
# perc.groupby('treatment').under_net.mean()
# perc['nn'] = perc.apply(lambda r: max(0, (250 - r.tot)) * r.under_net, 1)
# perc.groupby('treatment').nn.sum()
# dd.merge(treatment_assignment).groupby('treatment').under_net.sum()

In [ ]:
# saturated = perc[perc.non_target > 175].disthash
# audienced = perc[(perc.perc < .3) & ((perc.disthash.isin(saturated) | (perc.price < 30000)))].disthash
# cities['creative_group'] = 'hindi'
# cities.loc[cities.disthash.isin(saturated), 'creative_group'] = 'exclusions'
# cities['include_audience'] = False
# cities.loc[cities.disthash.isin(audienced), 'include_audience'] = True

In [ ]:
tot = 196952 + 17682 + 40222 + 89997 + 113061 + 50145 + 141910 + 4255509 + 46000 

In [ ]:
tot / 70